In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import scipy.stats as st
from datetime import datetime,date,timedelta
import time
import collections

In [2]:
# 定义的是核心按钮

jk_appid=['jkwx', 'jkand', 'jkand2', 'jkandroid', 'jkios', 'yzand1', 'jkiosspeed', 'jkiossolar','jkioslunar',
         'jkmjand_1','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd','jkiossjdjk',
         'jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkandjqb','jkandxedk','jkandpphd','jkori','jkoriand2','jkoriios']
button=['jk_acb_01_02','jk_acb_09_02','jk_acb_09_01','jk_acb_03_02','jk_acb_04_02','jk_acb_109_2','jk_acb_110_1']
button_dict={'jk_acb_01_02':'register_complete_by_ugid','jk_acb_09_02':'bankcard_addcard_click_cnt','jk_acb_09_01':'bankcard_creditcard_apply_click_cnt',
            'jk_acb_03_02':'addcard_complete_cnt','jk_acb_04_02':'addcreditcard_complete_cnt','jk_acb_109_2':'myinfo_signout_cnt',
            'jk_acb_110_1':'myinfo_signin_abnormal_alert_cnt'}

In [3]:
def mergeData(scr,tmp):
    data=pd.merge(scr, tmp,how = 'left', on = 'ugid')
    return data

In [4]:
def getCommonSDK(df):
    '''
    各核心按钮的点击次数
    '''
    result=df.groupby('ugid').eleId.nunique().reset_index()
    for item in button:
        tmp=df[df['eleId'] == item].groupby('ugid').size().reset_index().rename(columns={0:button_dict[item]})
        result=mergeData(result,tmp)
    result=result.drop('eleId',axis=1)
    return result    

In [6]:
# 读取数据

acq03=pd.read_csv("./data_mid/20171122_data_acq03.201706_201708.csv",escapechar = '\\',encoding = 'utf-8', 
                  error_bad_lines=False, dtype = {'utime':np.str, 'ctime':np.str, 'sessionId':np.str})

In [7]:
# 去重

acq03.drop_duplicates(inplace=True)
acq03.shape

(138321, 11)

In [8]:
acq03.head()

,ugid,zuid,appid,utime,ctime,ch,clickTime,eleId,path,remote_addr,sessionId
0,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178233.449,jku0000011,1.497178e+09,jk_acb_102_12,/index.html,223.104.15.252,1497178233449
1,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178289.927,NaN,1.497178e+09,jk_acb_02_01,/card/certification.html,223.104.15.252,1497178289927
2,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178289.927,NaN,1.497178e+09,jk_acb_02_02,/card/certification.html,223.104.15.252,1497178289927
3,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178302.468,NaN,1.497178e+09,jk_acb_16_05,/newlist/,223.104.15.252,1497178302468
4,57566a7f-a04a-4725-8004-e631b423d9a1,02cef52e3947d333b7b09a28f75788da|123456789002,jkwx,NaN,1496686747.55,NaN,1.496687e+09,jk_acb_101_04,/index.html,58.20.74.220,1496686747550


In [9]:
# 清理时间 & 时间筛选 & appid筛选
acq03=acq03[(acq03.sessionId==acq03.sessionId) & (acq03['sessionId']!='null')]
acq03=acq03[(acq03.ugid==acq03.ugid) & (acq03['ugid']!='null')]
acq03=acq03[acq03.appid.isin(jk_appid)]
acq03.shape

(100623, 11)

In [10]:
# 提取时间、格式转换
converDate=lambda x:datetime.fromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S')
acq03['date']=acq03.sessionId.apply(converDate)
acq03['date']=[datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in acq03['date']]

In [11]:
# 提取申请时间
label = pd.read_csv('./data_raw/APP_5K_20171120_ADD_UGID.csv')
# label = label.drop(['month','user_gid','last_decision_dt','SamplingWeight'],axis=1).rename(columns={'create_time':'last_decision_tm'})
label = label[['ugid','create_time']].rename(columns={'create_time':'last_decision_tm'})

In [12]:
# 申请时间格式转换
label.head()
label['last_decision_tm'] = label['last_decision_tm'].replace('DEC', '-12-',regex=True).\
replace('NOV', '-11-',regex=True).\
replace('OCT', '-10-',regex=True).\
replace('SEP', '-09-',regex=True).\
replace('AUG', '-08-',regex=True).\
replace('JUL', '-07-',regex=True).\
replace('JUN', '-06-',regex=True).\
replace('MAY', '-05-',regex=True).\
replace('APR', '-04-',regex=True).\
replace('MAR', '-03-',regex=True).\
replace('FEB', '-02-',regex=True).\
replace('JAN', '-01-',regex=True)

In [13]:
label.head()

,ugid,last_decision_tm
0,2ad5285a-c48b-466a-ab4b-591f41f10521,20-08-2017:00:07:22.967
1,3a89d1b3-07ca-4f90-b248-67cd19114949,12-08-2017:11:30:05.710
2,a8311a80-b7f8-4e2f-ae7e-3a3c4c1d8da4,04-08-2017:19:10:04.553
3,a3fa04ae-4665-4360-a62e-0faf5dc2e6eb,22-08-2017:18:52:00.110
4,b3e8d465-3de5-4564-9810-71f34be2b6ae,07-08-2017:00:21:11.143


In [14]:
label['jk_date']=[datetime.strptime(x,'%d-%m-%Y:%H:%M:%S.%f')for x in label.last_decision_tm]

In [15]:
acq03=pd.merge(acq03,label,on='ugid',how='right')

In [16]:
acq03.drop_duplicates(inplace=True)

In [17]:
acq03.shape

(103890, 14)

In [18]:
acq03.head()

,ugid,zuid,appid,utime,ctime,ch,clickTime,eleId,path,remote_addr,sessionId,date,last_decision_tm,jk_date
0,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178233.449,jku0000011,1.497178e+09,jk_acb_102_12,/index.html,223.104.15.252,1497178233449,2017-06-11 03:50:33,29-08-2017:11:40:07.480,2017-08-29 11:40:07.480
1,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178289.927,NaN,1.497178e+09,jk_acb_02_01,/card/certification.html,223.104.15.252,1497178289927,2017-06-11 03:51:29,29-08-2017:11:40:07.480,2017-08-29 11:40:07.480
2,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178289.927,NaN,1.497178e+09,jk_acb_02_02,/card/certification.html,223.104.15.252,1497178289927,2017-06-11 03:51:29,29-08-2017:11:40:07.480,2017-08-29 11:40:07.480
3,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1497178302.468,NaN,1.497178e+09,jk_acb_16_05,/newlist/,223.104.15.252,1497178302468,2017-06-11 03:51:42,29-08-2017:11:40:07.480,2017-08-29 11:40:07.480
4,1337df2a-7323-4d64-99a4-f2182c315c0d,0af1d4953ee606f4a42a0e25038ffcb6|123456789002,jkandroid,NaN,1501746548.598,jku0000011,1.501747e+09,jk_acb_101_01,/index.html,121.56.168.171,1501746548598,2017-08-03 00:49:08,29-08-2017:11:40:07.480,2017-08-29 11:40:07.480


In [19]:
%%time
# 特征提取入口

# 筛选时间窗口-30天
data_1m=acq03[(acq03.date+ timedelta(days=30) > acq03.jk_date) &  (acq03.date<acq03.jk_date) ].reset_index(drop=True)
allfeature=getCommonSDK(data_1m)
allfeature.columns=list(allfeature.columns[:1])+list('last1m_'+allfeature.columns[1:])

# 筛选时间窗口-7天
data_1w=acq03[(acq03.date+ timedelta(days=7) > acq03.jk_date) &  (acq03.date<acq03.jk_date) ].reset_index(drop=True)
tmp=getCommonSDK(data_1w)
tmp.columns=list(tmp.columns[:1])+list('last1w_'+tmp.columns[1:])
allfeature=mergeData(allfeature,tmp)

# 筛选时间窗口-1天
data_1d=acq03[(acq03.date+ timedelta(days=1) > acq03.jk_date) &  (acq03.date<acq03.jk_date) ].reset_index(drop=True)
tmp=getCommonSDK(data_1d)
tmp.columns=list(tmp.columns[:1])+list('last1d_'+tmp.columns[1:])
allfeature=mergeData(allfeature,tmp)

CPU times: user 372 ms, sys: 0 ns, total: 372 ms
Wall time: 366 ms


In [20]:
allfeature.fillna(-1,inplace=True)
allfeature.iloc[:,1:]=allfeature.iloc[:,1:].astype(int)

In [21]:
allfeature.to_csv('./data_last/20171123_feature_acq03_201706_201708.csv', index = False)

In [22]:
allfeature

,ugid,last1m_register_complete_by_ugid,last1m_bankcard_addcard_click_cnt,last1m_bankcard_creditcard_apply_click_cnt,last1m_addcard_complete_cnt,last1m_addcreditcard_complete_cnt,last1m_myinfo_signout_cnt,last1m_myinfo_signin_abnormal_alert_cnt,last1w_register_complete_by_ugid,last1w_bankcard_addcard_click_cnt,...,last1w_addcreditcard_complete_cnt,last1w_myinfo_signout_cnt,last1w_myinfo_signin_abnormal_alert_cnt,last1d_register_complete_by_ugid,last1d_bankcard_addcard_click_cnt,last1d_bankcard_creditcard_apply_click_cnt,last1d_addcard_complete_cnt,last1d_addcreditcard_complete_cnt,last1d_myinfo_signout_cnt,last1d_myinfo_signin_abnormal_alert_cnt
0,0003f621-0736-4d3f-a1b9-55b18adc4163,-1,-1,-1,-1,-1,3,-1,-1,-1,...,-1,2,-1,-1,-1,-1,-1,-1,-1,-1
1,0012385f-cde5-4397-a9fb-ffdc6a8662d3,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,006b04c8-c0ab-42a0-bf55-b236ec34d093,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,007747f5-041a-4e4b-a424-b732abb2e0fb,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0086f431-0f45-48d3-ace3-f76aa04e2bbf,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,00d3435a-b4df-49fb-b28c-6a25a57364ad,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,00de064b-016c-4135-8b4f-d4e2b9817884,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,0101efa7-6df5-4259-bb5c-507131e98c28,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,01210d14-fe7c-4283-bc47-232dc4527976,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,012a42af-bd92-459c-bf47-73c29fe7e01a,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
